## 파이썬 완정정복  메타클래스 및 메소드 클래스 정의


In [1]:
import sys

In [2]:
sys.version_info

sys.version_info(major=3, minor=6, micro=6, releaselevel='final', serial=0)

### 타입 메타 클래스로 일반 클래스 생성 

In [3]:
MyKlass = type.__new__(type, 'MyKlass', (object,), {})

In [4]:
type.__init__(MyKlass, 'MyKlass', (object,), {})

In [5]:
print(MyKlass)

<class '__main__.MyKlass'>


In [6]:
print(MyKlass.__dict__)

{'__module__': '__main__', '__dict__': <attribute '__dict__' of 'MyKlass' objects>, '__weakref__': <attribute '__weakref__' of 'MyKlass' objects>, '__doc__': None}


####  class 정의문을 이용해서 처리

In [7]:
class MyClass_(object) :
    pass

In [8]:
print(MyClass_)

<class '__main__.MyClass_'>


In [9]:
print(MyClass_.__dict__)

{'__module__': '__main__', '__dict__': <attribute '__dict__' of 'MyClass_' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass_' objects>, '__doc__': None}


### type 클래스에서 __call__ 로 클래스 정의

In [10]:
MyKlass_ = type.__call__(type, 'MyKlass_', (object,), {})

In [11]:
print(MyKlass_)
print(MyKlass_.__dict__)

<class '__main__.MyKlass_'>
{'__module__': '__main__', '__dict__': <attribute '__dict__' of 'MyKlass_' objects>, '__weakref__': <attribute '__weakref__' of 'MyKlass_' objects>, '__doc__': None}


### type 클래스 내의  prepare 처리

In [12]:
help(type.__prepare__)

Help on built-in function __prepare__:

__prepare__(...) method of builtins.type instance
    __prepare__() -> dict
    used to create the namespace for the class statement



In [13]:
def __init__(self,name) :
    self.name = name

In [14]:
a = type.__prepare__('AClass', (object,), __init__=__init__)
print(a)

{}


In [15]:
AClass = type.__call__(type, 'AClass', (object,), a)

In [16]:
print(AClass)
print(AClass.__dict__)

<class '__main__.AClass'>
{'__module__': '__main__', '__dict__': <attribute '__dict__' of 'AClass' objects>, '__weakref__': <attribute '__weakref__' of 'AClass' objects>, '__doc__': None}


### 메타클래스 내의 __prepare__ 알아보기

In [17]:
# 메타 클래스 정의
class Meta(type):
    def __new__(cls, name, bases, namespace):
        print("before name space ", namespace)
        result = type.__new__(cls, name, bases, namespace)
        
        return result
    
    @classmethod
    def __prepare__(metacls, name, bases):
        return {"abc": "abc"}
    

print(Meta)
print(Meta.__prepare__('aaa', (object, )))

<class '__main__.Meta'>
{'abc': 'abc'}


### 사용자 타입 클래스를 정의하고 실제 클래스 정의해서 사용하기 

In [18]:
import collections

#### 사용자 정의 메타클래스 정의 

In [19]:
class OrderedClass(type):

    @classmethod
    def __prepare__(metacls, name, bases, **kwds):
        return collections.OrderedDict()

    def __new__(cls, name, bases, namespace, **kwds):
        result = type.__new__(cls, name, bases, dict(namespace))
        result.members = tuple(namespace)
        return result



#### 사용자 클래스일 때 메타 클래스 할당

In [20]:

class A(metaclass=OrderedClass):
    def one(self): pass
    def two(self): pass
    def three(self): pass
    def four(self): pass

In [21]:
dir(A)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'four',
 'members',
 'one',
 'three',
 'two']

#### 내부적으로 생긴 네임스페이스를 확인할 수 있다. 

In [22]:
A.members

('__module__', '__qualname__', 'one', 'two', 'three', 'four')

### 외부에 함수를 정의해서 네임스페이스에 할당하기 

In [23]:
def howdy(self, you):
    print("Howdy, " + you)
    
MyList = type("MyList", (list,), dict(x=42, howdy=howdy))


In [24]:
ml = MyList()
ml.append("Camembert")
print(ml)

['Camembert']


In [25]:
print(MyList.__dict__)
print(ml.x)
ml.howdy("John")

{'x': 42, 'howdy': <function howdy at 0x000000000639F2F0>, '__module__': '__main__', '__dict__': <attribute '__dict__' of 'MyList' objects>, '__weakref__': <attribute '__weakref__' of 'MyList' objects>, '__doc__': None}
42
Howdy, John


In [26]:
print(ml.__class__)
print(ml.__class__.__class__)

<class '__main__.MyList'>
<class 'type'>


### 초기화 함수를 네임스페이스에 전달

In [27]:
# __init__함수 정의
def __init__(self, name, salary):
    self.name = name
    self.salary = salary
    
Person = type("Person", (object,), dict(__init__=__init__))


In [28]:
b = Person('dahl', 20000)
print(type(b))

for i in Person.__dict__:
    print(i)

<class '__main__.Person'>
__init__
__module__
__dict__
__weakref__
__doc__


### 메타클래스를 정의하고 내부 구조를 다시 알아보기

In [29]:
# 메타 클래스 정의
class Meta(type):
    def __new__(cls, name, bases, namespace):
        print("before name space ", namespace)
        result = type.__new__(cls, name, bases, namespace)
        
        return result
    
    @classmethod
    def __prepare__(metacls, name, bases):
        return {"abc": "abc"}
    

In [30]:
# class정의 및 metaclass 지정
class A(metaclass=Meta):
    def __init__(self):
        pass
    
    def one(self): pass
    def two(self): pass
    def three(self): pass
    def four(self): pass
    


before name space  {'abc': 'abc', '__module__': '__main__', '__qualname__': 'A', '__init__': <function A.__init__ at 0x000000000639F0D0>, 'one': <function A.one at 0x000000000639F048>, 'two': <function A.two at 0x00000000065ABF28>, 'three': <function A.three at 0x00000000065ABEA0>, 'four': <function A.four at 0x00000000065ABE18>}


In [31]:
a = A()
print("the appropriate metaclass is determined ==> ")
print(type(A))
print("the class namespace is prepared ===>")
print(A.__dict__)

the appropriate metaclass is determined ==> 
<class '__main__.Meta'>
the class namespace is prepared ===>
{'abc': 'abc', '__module__': '__main__', '__init__': <function A.__init__ at 0x000000000639F0D0>, 'one': <function A.one at 0x000000000639F048>, 'two': <function A.two at 0x00000000065ABF28>, 'three': <function A.three at 0x00000000065ABEA0>, 'four': <function A.four at 0x00000000065ABE18>, '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__doc__': None}


### 사용자 메소드 클래스를 정의하기

In [32]:
class Method :
    def __init__(self,func) :
        self._func = func
        
    def __call__(self, *args, **kwargs) :
        return self._func(self,*args, **kwargs)
    
    def method(self) :
        return 100

#### 내부 클래스를 사용자 메소드로 데코레이터 처리하기

In [33]:
class A :
    @Method
    def a(self) :
        return 10

In [34]:
A.__dict__

mappingproxy({'__dict__': <attribute '__dict__' of 'A' objects>,
              '__doc__': None,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              'a': <__main__.Method at 0x6395be0>})

In [35]:
bb = A()

#### 메소드 실행하기

In [36]:
bb.a

In [37]:
bb.a()

10

#### 메소드 내의 네임스페이스 확인하기

In [38]:
bb.a.__dict__

{'_func': <function __main__.A.a>}

#### 메소드 내의 메소드 처리

In [39]:
dir(bb.a)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_func',
 'method']

In [40]:
bb.a.method()

100